In [2]:
import numpy  as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt 
%matplotlib inline

import seaborn as sns

import json 

import warnings
warnings.filterwarnings(action='ignore')

from datetime import date, datetime, timedelta
from dateutil.parser import parse


# 한글 폰트 문제 해결
import platform

from matplotlib import font_manager, rc

if platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~') 


# 차트 축 <- 음수 부호 지원
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False


# crawling
from bs4 import BeautifulSoup
from urllib.request import urlopen , urlretrieve
from urllib.error   import HTTPError
from urllib.error   import URLError

import requests 
import re

from selenium import webdriver

from time    import sleep , time 
from random  import randint
from IPython.core.display import clear_output

# 비정형 디비 
import pymongo as mongo

print('numpy version  - ' , np.__version__)
print('pandas version - ' , pd.__version__)

numpy version  -  1.20.3
pandas version -  1.3.4


#### 크롤링과 몽고디비 연결
- http://www.cine21.com/rank/person/content
- form data : section=actor&period_start=2020-04&gender=f&page=1

In [3]:
print('connection - ')
conn = mongo.MongoClient()
conn

connection - 


MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
print('new database create - ')
cine_db = conn.cine21
cine_db

print()
print('collection create - table - ') 
cine_collection = cine_db.cine_collection
cine_collection

new database create - 

collection create - table - 


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'cine21'), 'cine_collection')

- 크롤링 준비 requests

In [5]:
print('case01 - bs ')
print()

url = 'http://www.cine21.com/rank/person/content'
queryString = dict()
queryString['section']      = 'actor'
queryString['period_start'] = '2022-03'
queryString['gender']       = 'all'
queryString['page']         = '1'

response = requests.post(url , queryString) 
response
soup = BeautifulSoup(response.content , 'html.parser')
soup


case01 - bs 



 <ul class="people_list">
<li class="people_li">
<a href="/db/person/info/?person_id=2262"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/person/2020/0812/2262[X145,145].jpg" target="_blank"/></a>
<div class="name"><a href="/db/person/info/?person_id=2262">최민식(1편)</a></div>
<ul class="num_info">
<li><span class="tit">흥행지수</span><strong>162,280</strong></li>
<!--
						<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>
						-->
</ul>
<!-- 영화포스터는 최대 5개까지만 -->
<ul class="mov_list">
<li>
<a href="/movie/info/?movie_id=55829">
<img alt="" class="thumb" src="https://image.cine21.com/resize/cine21/poster/2022/0211/55829_6205b39abb231[X85,120].jpg" target="_blank"/>
<span>이상한 나라의 수학자</span>
</a>
</li>
</ul>
<!-- 순위 --><span class="grade">1</span>
</li>
<li class="people_li">
<a href="/db/person/info/?person_id=29321"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/person/2020/0122/11_55_

In [6]:
print('case02 - sl - xpath')

path = './driver/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get('http://www.cine21.com/rank/person/')


print('bs를 이용한 배우이름만 추출 - ')
print()
lis = soup.select('.people_list li div.name')
for li in lis :
    print( re.sub('\(\w*\)' , '' , li.get_text() ))

case02 - sl - xpath
bs를 이용한 배우이름만 추출 - 

최민식
김동휘
정우
박해준
김갑수
최무성
박병은


In [7]:
print('배우이름, 흥행지수, 출연영화 크롤링 - ')
print()
actors  = soup.select('.people_list li div.name')
hits    = soup.select('ul.num_info > li > strong')
movies  = soup.select('ul.mov_list')

for idx , actor in enumerate(actors) :
    print('배우이름 : ' , re.sub('\(\w*\)' , '' , actor.get_text() ) )
    print('흥행지수 : ' , hits[idx].get_text().replace(',' , ''))
    # print( movies[idx].select('li a span')[0].get_text() )
    titles = movies[idx].select('li a span')
    title_lst = []
    for title in titles : 
        title_lst.append( title.get_text() )
    print('출연영화 : ' , title_lst)
    print('*' * 50)    

배우이름, 흥행지수, 출연영화 크롤링 - 

배우이름 :  최민식
흥행지수 :  162280
출연영화 :  ['이상한 나라의 수학자']
**************************************************
배우이름 :  김동휘
흥행지수 :  129824
출연영화 :  ['이상한 나라의 수학자']
**************************************************
배우이름 :  정우
흥행지수 :  111714
출연영화 :  ['뜨거운 피']
**************************************************
배우이름 :  박해준
흥행지수 :  97395
출연영화 :  ['이상한 나라의 수학자', '4등']
**************************************************
배우이름 :  김갑수
흥행지수 :  89371
출연영화 :  ['뜨거운 피']
**************************************************
배우이름 :  최무성
흥행지수 :  71108
출연영화 :  ['뜨거운 피', '배니싱: 미제사건', '4등']
**************************************************
배우이름 :  박병은
흥행지수 :  64912
출연영화 :  ['이상한 나라의 수학자']
**************************************************


In [8]:
print('수집 데이터기반 dict 만들어서 디비에 입력하기 - ') 
print('배우이름, 흥행지수, 출연영화, 랭킹 크롤링 - ')
print('다른 이름, 직업, 생년월일, 성별, 홈페이지, 신장/체중, 학교, 취미  - ')
print()

print()
actors   = soup.select('.people_list li div.name')
hits     = soup.select('ul.num_info > li > strong')
movies   = soup.select('ul.mov_list')

rankings = soup.select('li.people_li > span.grade')

actors_info_list = list()

for idx , actor in enumerate(actors) :
    
    detail_dict = dict()
    
    # print('배우이름 : ' , re.sub('\(\w*\)' , '' , actor.get_text() ) )
    name = re.sub('\(\w*\)' , '' , actor.get_text() ) 
    
    # print('흥행지수 : ' , hits[idx].get_text().replace(',' , ''))
    hit = hits[idx].get_text().replace(',' , '')
    # print( movies[idx].select('li a span')[0].get_text() )
    
    titles = movies[idx].select('li a span')
    title_lst = []
    for title in titles : 
        title_lst.append( title.get_text() )
    
    # print('출연영화 : ' , title_lst)
    title = title_lst
    # print('랭킹     : ' , rankings[idx].get_text() )
    ranking = rankings[idx].get_text()
    # print( actor.a['href'] )
    actor_link = 'http://www.cine21.com'+actor.select_one('a').attrs['href']
    # print( actor_link )
    
    actor_detail = requests.get( actor_link ) 
    actor_detail_soup = BeautifulSoup( actor_detail.content , 'html.parser')
    # print( actor_detail_soup )
    
    detail_info = actor_detail_soup.select_one('ul.default_info')
    # print( detail_info )
    detail_info_lis = detail_info.select('li') 
    
    
    detail_dict['배우이름'] = name
    detail_dict['흥행지수'] = hit
    detail_dict['출연영화'] = title
    detail_dict['랭킹'] = ranking
    
    
    for li in detail_info_lis : 
        # print( li.select('span.tit'))
        detail_key = li.select('span.tit')[0].get_text()
        
        # print( re.sub('<span.*?>*?</span>' , '' , str(li) ) )
        detail_value = re.sub('<span.*?>.*?</span>' , '' , str(li) ) 
        detail_value = re.sub('<.*?>' , '' , detail_value ) 
        # print( re.sub('<span.*?>*?</span>' , '' , detail_value ) )
        
        detail_dict[detail_key] = detail_value
    
    print(detail_dict)
    print('*' * 50)  
    actors_info_list.append(detail_dict)

수집 데이터기반 dict 만들어서 디비에 입력하기 - 
배우이름, 흥행지수, 출연영화, 랭킹 크롤링 - 
다른 이름, 직업, 생년월일, 성별, 홈페이지, 신장/체중, 학교, 취미  - 


{'배우이름': '최민식', '흥행지수': '162280', '출연영화': ['이상한 나라의 수학자'], '랭킹': '1', '다른 이름': '崔岷植', '직업': '배우', '생년월일': '1962-04-27', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/cjesminsik\n', '신장/체중': '177cm, 70kg', '학교': '동국대학교 연극영화', '취미': '공연 관람, 수집, 그림그리기, 골프'}
**************************************************
{'배우이름': '김동휘', '흥행지수': '129824', '출연영화': ['이상한 나라의 수학자'], '랭킹': '2', '직업': '배우', '성별': '남'}
**************************************************
{'배우이름': '정우', '흥행지수': '111714', '출연영화': ['뜨거운 피'], '랭킹': '3', '원어명': '김정국', '직업': '배우', '생년월일': '1981-01-14', '성별': '남', '홈페이지': '\nhttps://twitter.com/jungwoo109\n', '신장/체중': '180cm, 70kg', '학교': '서울예술대학 영화과', '취미': '운동(아크로바티, 복싱, 태권도), 노래부르기'}
**************************************************
{'배우이름': '박해준', '흥행지수': '97395', '출연영화': ['이상한 나라의 수학자', '4등'], '랭킹': '4', '직업': '배우', '생년월일': '1976-06-14', '성별': '남'}
*************************

In [9]:
print('여러페이지에 대한 상세정보 추출 - ')
print('수집 데이터기반 dict 만들어서 디비에 입력하기 - ') 
print('배우이름, 흥행지수, 출연영화, 랭킹 크롤링 - ')
print('다른 이름, 직업, 생년월일, 성별, 홈페이지, 신장/체중, 학교, 취미  - ')
print()

print()

url = 'http://www.cine21.com/rank/person/content'
queryString = dict()
queryString['section']      = 'actor'
queryString['period_start'] = '2022-03'
queryString['gender']       = 'all'



actors_info_list = list()

for page in range(1 , 3) :
    
    queryString['page'] = page
    
    response = requests.post(url , queryString) 
    soup = BeautifulSoup(response.content , 'html.parser')
    
    
    actors   = soup.select('.people_list li div.name')
    hits     = soup.select('ul.num_info > li > strong')
    movies   = soup.select('ul.mov_list')

    rankings = soup.select('li.people_li > span.grade')
        
    for idx , actor in enumerate(actors) :

        detail_dict = dict()

        # print('배우이름 : ' , re.sub('\(\w*\)' , '' , actor.get_text() ) )
        name = re.sub('\(\w*\)' , '' , actor.get_text() ) 

        # print('흥행지수 : ' , hits[idx].get_text().replace(',' , ''))
        hit = hits[idx].get_text().replace(',' , '')
        # print( movies[idx].select('li a span')[0].get_text() )

        titles = movies[idx].select('li a span')
        title_lst = []
        for title in titles : 
            title_lst.append( title.get_text() )

        # print('출연영화 : ' , title_lst)
        title = title_lst
        # print('랭킹     : ' , rankings[idx].get_text() )
        ranking = rankings[idx].get_text()
        # print( actor.a['href'] )
        actor_link = 'http://www.cine21.com'+actor.select_one('a').attrs['href']
        # print( actor_link )

        actor_detail = requests.get( actor_link ) 
        actor_detail_soup = BeautifulSoup( actor_detail.content , 'html.parser')
        # print( actor_detail_soup )

        detail_info = actor_detail_soup.select_one('ul.default_info')
        # print( detail_info )
        detail_info_lis = detail_info.select('li') 


        detail_dict['배우이름'] = name
        detail_dict['흥행지수'] = hit
        detail_dict['출연영화'] = title
        detail_dict['랭킹'] = ranking


        for li in detail_info_lis : 
            # print( li.select('span.tit'))
            detail_key = li.select('span.tit')[0].get_text()

            # print( re.sub('<span.*?>*?</span>' , '' , str(li) ) )
            detail_value = re.sub('<span.*?>.*?</span>' , '' , str(li) ) 
            detail_value = re.sub('<.*?>' , '' , detail_value ) 
            # print( re.sub('<span.*?>*?</span>' , '' , detail_value ) )

            detail_dict[detail_key] = detail_value

        print(detail_dict)
        print('*' * 50)  
        actors_info_list.append(detail_dict)

여러페이지에 대한 상세정보 추출 - 
수집 데이터기반 dict 만들어서 디비에 입력하기 - 
배우이름, 흥행지수, 출연영화, 랭킹 크롤링 - 
다른 이름, 직업, 생년월일, 성별, 홈페이지, 신장/체중, 학교, 취미  - 


{'배우이름': '최민식', '흥행지수': '162280', '출연영화': ['이상한 나라의 수학자'], '랭킹': '1', '다른 이름': '崔岷植', '직업': '배우', '생년월일': '1962-04-27', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/cjesminsik\n', '신장/체중': '177cm, 70kg', '학교': '동국대학교 연극영화', '취미': '공연 관람, 수집, 그림그리기, 골프'}
**************************************************
{'배우이름': '김동휘', '흥행지수': '129824', '출연영화': ['이상한 나라의 수학자'], '랭킹': '2', '직업': '배우', '성별': '남'}
**************************************************
{'배우이름': '정우', '흥행지수': '111714', '출연영화': ['뜨거운 피'], '랭킹': '3', '원어명': '김정국', '직업': '배우', '생년월일': '1981-01-14', '성별': '남', '홈페이지': '\nhttps://twitter.com/jungwoo109\n', '신장/체중': '180cm, 70kg', '학교': '서울예술대학 영화과', '취미': '운동(아크로바티, 복싱, 태권도), 노래부르기'}
**************************************************
{'배우이름': '박해준', '흥행지수': '97395', '출연영화': ['이상한 나라의 수학자', '4등'], '랭킹': '4', '직업': '배우', '생년월일': '1976-06-14', '성별': '남'}
****

In [ ]:
print('위의 bs를 selenium의 XPATH를 바꿔보기')
print('click(),xpath()')
print('여러 페이지에 대한 상세정보 추출')
print('수집 데이터기반 dict 만들어서 디비에 입력하기')
print('배우이름, 흥행지수, 출연영화, 랭킹 크롤링')
print('다른이름, 직업, 생년월일, 성별, 홈페이지, 신장/체중, 학교, 취미 크롤링')
print()
driver.get('http://www.cine21.com/rank/person/')
actor_info_list = list()
for i in range(1,3):
    driver.find_element_by_xpath('//*[@id="rank_holder"]/div/div/a[{}]'.format(i)).click()
    print('page :',i)
    lists = driver.find_elements_by_xpath('//*[@id="rank_holder"]/ul/li')
    for idx in range(len(lists)):
        actor_info = dict()
        print(i)
        print(idx)
        sleep(3)
        rank = driver.find_element_by_xpath('//*[@id="rank_holder"]/ul/li[{}+1]/span'.format(idx)).text
        name = re.sub('\(\w*\)','',driver.find_element_by_xpath('//*[@id="rank_holder"]/ul/li[{}+1]/div/a'.format(idx)).text)
        hit = driver.find_element_by_xpath('//*[@id="rank_holder"]/ul/li[{}+1]/ul[1]/li/strong'.format(idx)).text.replace(',','')
        title_list = [title.text for title in driver.find_elements_by_xpath('//*[@id="rank_holder"]/ul/li[{}+1]/ul[2]/li'.format(idx))]
        actor_info['rank'] = rank
        actor_info['name'] = name
        actor_info['hit'] = hit
        actor_info['title_list'] = title_list
        driver.find_element_by_xpath('//*[@id="rank_holder"]/ul/li[{}+1]/div/a'.format(idx)).click()
        for info in driver.find_elements_by_xpath('//*[@id="info_content"]/div[1]/ul/li'):
            detail_sub = info.find_element_by_xpath('span').text
            detail_val = re.sub(info.find_element_by_xpath('span').text,'',str(info.text))
            actor_info[detail_sub] = detail_val
        print('='*117)
        actor_info_list.append(actor_info)
        driver.back()
        sleep(3)
        driver.find_element_by_xpath('//*[@id="rank_holder"]/div/div/a[{}]'.format(i)).click()
        lists = driver.find_elements_by_xpath('//*[@id="rank_holder"]/ul/li')